# Forza

## Input Line and Line List Functions

In [ ]:
def forza_section(input_line):
    section_replace_dict = {
        "=": ""
    }
    for old, new in section_replace_dict.items():
        input_line = input_line.replace(old, new)
    input_line = input_line.strip()
    return input_line

In [ ]:
def forza_manufacturer(input_line):
    html_comment_replace_dict = {
        "<!--": "",
        "-->": ""
    }
    for old, new in html_comment_replace_dict.items():
        input_line = input_line.replace(old, new)
    input_line = input_line.strip("- ")
    return ( 
        forza_manufacturer_translation_dict[input_line].strip()
        if input_line in forza_manufacturer_translation_dict.keys() 
        else input_line
    )

In [ ]:
def forza_line_list_from_table_section(input_line, section):
    line_list = []
    if "<nowiki>|</nowiki>" not in input_line:
        input_line = re.sub(html_tag_cleaner, "", input_line)
    for item in ["{{", "}}"]:
        input_line = input_line.replace(item, "")
    if "<nowiki>|</nowiki>" in input_line:
        input_line = input_line.split("<nowiki>|</nowiki>")
        for i in range(len(input_line)):
            input_line[i] = input_line[i].split("|")
        new_string = f"{input_line[0][-1]} | {input_line[1][0]}"
        line_list.extend(input_line[0][:-1])
        line_list.extend([new_string])
        line_list.extend(input_line[1][1:])
        line_list = [line_list]
    else:
        line_list = re.split(
            "|".join(["\[\[", "\]\]"]), 
            input_line
        )
        for i in range(len(line_list)):
            line_list[i] = line_list[i].split("|")
    if game == "Forza Horizon 4":
        line_list[2][1] = [line_list[2][1]]
        if len(line_list) > 3:
            excess_sublists = line_list[3:]
            line_list[2][1].append(line_list[2][-1])
            line_list[2].remove(line_list[2][-1])
            for i in range(len(excess_sublists) - 1):
                line_list[2][1].extend(line_list[i + 3])
                line_list.remove(line_list[i + 3])
            line_list[2].extend(line_list[3][1:])
            line_list.remove(line_list[3])
    if game == "Forza Horizon 5":
        decrem_equals = 0
        amount_to_add = 0
        for i in range(len(line_list[0])):
            if "=" in line_list[0][i]:
                split_item = line_list[0][i].split("=")
                try:
                    max_line_list_length = int(split_item[0]) + 1
                    line_list[0][i] = split_item[1]
                    amount_to_add = max_line_list_length - len(line_list[0])
                    decrem_equals -= 1
                except:
                    continue
        if decrem_equals < 0:
            for i in range(amount_to_add):
                line_list[0].insert(decrem_equals, "")
    return line_list

In [ ]:
def forza_line_list_from_list_section(input_line, section):
    if section in ["Traffic", "Autovista-Only Cars"] and input_line.startswith("*"):
        line_list = re.split(
            "|".join(["\*", "\[\[", "\]\]"]), 
            input_line
        )
        line_list[2] = line_list[2].split("|")
    return line_list

## Data Extraction

In [ ]:
vehicle_data_list = []

In [ ]:
def forza_data_extraction(game, data):
    section = ""
    manufacturer = ""
    for input_line in data:
        # Skip Cases
        skip_cases = [
            input_line.startswith(item)
            for item in forza_skip_startswith_list
        ]
        if any(skip_cases):
            continue
        # Section
        if input_line.startswith("=="):
            section = forza_section(input_line)
            if game in enable_print_list:
                print("\t", section)
        # Manufacturer
        if input_line.startswith("<!--"):
            manufacturer = forza_manufacturer(input_line)
            if game in enable_print_list:
                print("\t\t", manufacturer)
        # Vehicle 
        if input_line.startswith("{{CarList"):
            line_list = forza_line_list_from_table_section(input_line, section)
            if section == "Unobtainable":
                manufacturer = difflib.get_close_matches(
                    " ".join(line_list[0][3].strip()[0:2])
                    if line_list[0][3].strip() != ""
                    else " ".join(line_list[0][2].strip()[0:2]),  
                    forza_unobtainable_vehicle_manufacturers_dict[f"{section} {game}"], 
                    n=1, cutoff=0
                )[0]
            if game in enable_print_list:
                print("\t\t\t", line_list)
            # Formatting for each game
            match game:
                case "Forza Motorsport 1":
                    vehicle_data_list.append([
                        game, #"Game"
                        manufacturer, #"Manufacturer"
                        forza_vehicle_name(game, line_list[1]), #"Vehicle Name" 
                        forza_vehicle_name(game, line_list[1], alt = True), #"Vehicle Name Alternative"
                        forza_year(line_list[0][1]), #"Year"
                        "Base Game", #"How to Obtain"
                        forza_value(line_list[2][1]), #"Value" 
                        forza_pi_class( #"Performance Index Class"
                            game, 
                            pi_class_string = line_list[2][2]
                        ), 
                        None, #"Performance Index Number"
                        None, #"Type"
                        None, #"Era"
                        None, #"Country"
                        None, #"Rarity"
                        forza_rating(line_list[2][7]), #"Rarity Rating"
                        forza_rating(line_list[2][3]), #"Speed Rating"
                        forza_rating(line_list[2][4]), #"Acceleration Rating"
                        forza_rating(line_list[2][5]), #"Braking Rating"
                        forza_rating(line_list[2][6]), #"Handling Rating"
                        None, #"Launch Rating"
                        None, #"Offroad Rating"
                        forza_link(line_list[1][0]) #"Link"
                    ])
                case "Forza Motorsport 2":
                    vehicle_data_list.append([
                        game, #"Game"
                        manufacturer, #"Manufacturer"
                        forza_vehicle_name(game, line_list[1]), #"Vehicle Name" 
                        forza_vehicle_name(game, line_list[1], alt = True), #"Vehicle Name Alternative"
                        forza_year(line_list[0][2]), #"Year"
                        forza_how_to_obtain(game, line_list[0][1]), #"How to Obtain"
                        forza_value(line_list[2][1]), #"Value"
                        forza_pi_class( #"Performance Index Class" 
                            game, 
                            pi_class_string = line_list[2][2], 
                            pi_number = line_list[2][3]
                        ), 
                        forza_pi_number(line_list[2][3]), #"Performance Index Number"
                        None, #"Type"
                        None, #"Era"
                        None, #"Country"
                        None, #"Rarity"
                        forza_rating(line_list[2][8]), #"Rarity Rating"
                        forza_rating(line_list[2][4]), #"Speed Rating"
                        forza_rating(line_list[2][5]), #"Acceleration Rating"
                        forza_rating(line_list[2][6]), #"Braking Rating"
                        forza_rating(line_list[2][7]), #"Handling Rating"
                        None, #"Launch Rating"
                        None, #"Offroad Rating"
                        forza_link(line_list[1][0]) #"Link"
                    ])
                case "Forza Motorsport 3":
                    vehicle_data_list.append([
                        game, #"Game"
                        manufacturer, #"Manufacturer"
                        forza_vehicle_name(game, line_list[1]), #"Vehicle Name" 
                        forza_vehicle_name(game, line_list[1], alt = True), #"Vehicle Name Alternative"
                        forza_year(line_list[0][1]), #"Year"
                        forza_how_to_obtain(game, line_list[2][0]), #"How to Obtain"
                        forza_value(line_list[2][1]), #"Value"
                        forza_pi_class( #"Performance Index Class"  
                            game, 
                            pi_number = line_list[2][2]
                        ),
                        forza_pi_number(line_list[2][2]), #"Performance Index Number"
                        None, #"Type"
                        None, #"Era"
                        None, #"Country"
                        None, #"Rarity"
                        None, #"Rarity Rating"
                        forza_rating(line_list[2][3]), #"Speed Rating"
                        forza_rating(line_list[2][5]), #"Acceleration Rating"
                        forza_rating(line_list[2][7]), #"Braking Rating"
                        forza_rating(line_list[2][4]), #"Handling Rating"
                        forza_rating(line_list[2][6]), #"Launch Rating"
                        None, #"Offroad Rating"
                        forza_link(line_list[1][0]) #"Link"
                    ])
                case "Forza Motorsport 4":
                    vehicle_data_list.append([
                        game, #"Game"
                        manufacturer, #"Manufacturer"
                        forza_vehicle_name(game, line_list[1]), #"Vehicle Name" 
                        forza_vehicle_name(game, line_list[1], alt = True), #"Vehicle Name Alternative"
                        forza_year(line_list[0][1]), #"Year"
                        forza_how_to_obtain(game, line_list[2][0]), #"How to Obtain" 
                        forza_value(line_list[2][2]), #"Value"
                        forza_pi_class( #"Performance Index Class" 
                            game, 
                            pi_number = line_list[2][4]
                        ), 
                        forza_pi_number(line_list[2][4]), #"Performance Index Number"
                        None, #"Type"
                        None, #"Era"
                        None, #"Country"
                        None, #"Rarity"
                        None, #"Rarity Rating"
                        forza_rating(line_list[2][5]), #"Speed Rating"
                        forza_rating(line_list[2][7]), #"Acceleration Rating"
                        forza_rating(line_list[2][9]), #"Braking Rating"
                        forza_rating(line_list[2][6]), #"Handling Rating"
                        forza_rating(line_list[2][8]), #"Launch Rating"
                        None, #"Offroad Rating"
                        forza_link(line_list[1][0]) #"Link"
                    ])
                case "Forza Motorsport 5":
                    vehicle_data_list.append([
                        game, #"Game"
                        manufacturer, #"Manufacturer"
                        forza_vehicle_name(game, line_list[1]), #"Vehicle Name" 
                        forza_vehicle_name(game, line_list[1], alt = True), #"Vehicle Name Alternative" 
                        forza_year(line_list[0][1]), #"Year"
                        forza_how_to_obtain(game, line_list[2][0]), #"How to Obtain"
                        forza_value(line_list[2][1]), #"Value"
                        forza_pi_class( #"Performance Index Class" 
                            game, 
                            pi_number = line_list[2][3]
                        ),
                        forza_pi_number(line_list[2][3]), #"Performance Index Number"
                        None, #"Type"
                        None, #"Era"
                        None, #"Country"
                        None, #"Rarity"
                        None, #"Rarity Rating"
                        forza_rating(line_list[2][4]), #"Speed Rating"
                        forza_rating(line_list[2][7]), #"Acceleration Rating"
                        forza_rating(line_list[2][8]), #"Braking Rating"
                        forza_rating(line_list[2][5]), #"Handling Rating"
                        forza_rating(line_list[2][6]), #"Launch Rating"
                        None, #"Offroad Rating"
                        forza_link(line_list[1][0]) #"Link"
                    ])
                case "Forza Motorsport 6":
                    vehicle_data_list.append([
                        game, #"Game"
                        manufacturer, #"Manufacturer"
                        forza_vehicle_name(game, line_list[1]), #"Vehicle Name" 
                        forza_vehicle_name(game, line_list[1], alt = True), #"Vehicle Name Alternative"
                        forza_year(line_list[0][2]), #"Year"
                        forza_how_to_obtain(game, line_list[0][1]), #"How to Obtain"
                        forza_value(line_list[2][1]), #"Value"
                        forza_pi_class( #"Performance Index Class" 
                            game, 
                            pi_number = line_list[2][3]
                        ), 
                        forza_pi_number(line_list[2][3]), #"Performance Index Number"
                        None, #"Type"
                        None, #"Era"
                        None, #"Country"
                        None, #"Rarity"
                        None, #"Rarity Rating"
                        forza_rating(line_list[2][4]), #"Speed Rating"
                        forza_rating(line_list[2][6]), #"Acceleration Rating"
                        forza_rating(line_list[2][7]), #"Braking Rating"
                        forza_rating(line_list[2][5]), #"Handling Rating"
                        None, #"Launch Rating"
                        None, #"Offroad Rating"
                        forza_link(line_list[1][0]) #"Link"
                    ])
                case "Forza Motorsport 7":
                    vehicle_data_list.append([
                        game, #"Game"
                        manufacturer, #"Manufacturer"
                        forza_vehicle_name(game, line_list[1]), #"Vehicle Name" 
                        forza_vehicle_name(game, line_list[1], alt = True), #"Vehicle Name Alternative"
                        forza_year(line_list[0][1]), #"Year"
                        forza_how_to_obtain(game, line_list[2][10:]), #"How to Obtain"
                        forza_value(line_list[2][4]), #"Value"
                        forza_pi_class( #"Performance Index Class" 
                            game, 
                            pi_number = line_list[2][9]
                        ), 
                        forza_pi_number(line_list[2][9]), #"Performance Index Number"
                        forza_type(game, line_list[2][1]), #"Type"
                        None, #"Era"
                        None, #"Country"
                        forza_rarity(game, line_list[2][2]), #"Rarity"
                        None, #"Rarity Rating"
                        forza_rating(line_list[2][5]), #"Speed Rating"
                        forza_rating(line_list[2][7]), #"Acceleration Rating"
                        forza_rating(line_list[2][8]), #"Braking Rating"
                        forza_rating(line_list[2][6]), #"Handling Rating"
                        None, #"Launch Rating"
                        None, #"Offroad Rating"
                        forza_link(line_list[1][0]) #"Link"
                    ])
                case "Forza Horizon 1":
                    vehicle_data_list.append([
                        game, #"Game"
                        manufacturer, #"Manufacturer"
                        forza_vehicle_name(game, line_list[1]), #"Vehicle Name" 
                        forza_vehicle_name(game, line_list[1], alt = True), #"Vehicle Name Alternative"
                        forza_year(line_list[0][1]), #"Year"
                        forza_how_to_obtain(game, line_list[0][1]), #"How to Obtain"
                        forza_value(line_list[2][2]), #"Value"
                        forza_pi_class( #"Performance Index Class" 
                            game, 
                            pi_number = line_list[2][3]
                        ), 
                        forza_pi_number(line_list[2][3]), #"Performance Index Number"
                        None, #"Type"
                        None, #"Era"
                        None, #"Country"
                        None, #"Rarity"
                        None, #"Rarity Rating"
                        forza_rating(line_list[2][4]), #"Speed Rating"
                        forza_rating(line_list[2][6]), #"Acceleration Rating"
                        forza_rating(line_list[2][8]), #"Braking Rating"
                        forza_rating(line_list[2][5]), #"Handling Rating"
                        forza_rating(line_list[2][7]), #"Launch Rating"
                        None, #"Offroad Rating"
                        forza_link(line_list[1][0]) #"Link"
                    ])
                case "Forza Horizon 2":
                    vehicle_data_list.append([
                        game, #"Game"
                        manufacturer, #"Manufacturer"
                        forza_vehicle_name(game, line_list[1]), #"Vehicle Name" 
                        forza_vehicle_name(game, line_list[1], alt = True), #"Vehicle Name Alternative"
                        forza_year(line_list[0][2]), #"Year"
                        forza_how_to_obtain(game, line_list[0][1]), #"How to Obtain"
                        forza_value(line_list[2][1]), #"Value"
                        forza_pi_class( #"Performance Index Class" 
                            game, 
                            pi_number = line_list[2][2]
                        ), 
                        forza_pi_number(line_list[2][2]), #"Performance Index Number"
                        None, #"Type"
                        None, #"Era"
                        None, #"Country"
                        None, #"Rarity"
                        None, #"Rarity Rating"
                        forza_rating(line_list[2][3]), #"Speed Rating"
                        forza_rating(line_list[2][5]), #"Acceleration Rating"
                        forza_rating(line_list[2][7]), #"Braking Rating"
                        forza_rating(line_list[2][4]), #"Handling Rating"
                        forza_rating(line_list[2][6]), #"Launch Rating"
                        None, #"Offroad Rating"
                        forza_link(line_list[1][0]) #"Link"
                    ])
                case "Forza Horizon 3":
                    vehicle_data_list.append([
                        game, #"Game"
                        manufacturer, #"Manufacturer"
                        forza_vehicle_name(game, line_list[1]), #"Vehicle Name" 
                        forza_vehicle_name(game, line_list[1], alt = True), #"Vehicle Name Alternative"
                        forza_year(line_list[0][1]), #"Year"
                        forza_how_to_obtain(game, line_list[2][9:]), #"How to Obtain"
                        forza_value(line_list[2][1]), #"Value"
                        forza_pi_class( #"Performance Index Class" 
                            game, 
                            pi_number = line_list[2][3]
                        ), 
                        forza_pi_number(line_list[2][3]), #"Performance Index Number"
                        forza_type(game, line_list[2][2]), #"Type"
                        None, #"Era"
                        None, #"Country"
                        None, #"Rarity"
                        None, #"Rarity Rating"
                        forza_rating(line_list[2][4]), #"Speed Rating"
                        forza_rating(line_list[2][6]), #"Acceleration Rating"
                        forza_rating(line_list[2][8]), #"Braking Rating"
                        forza_rating(line_list[2][5]), #"Handling Rating"
                        forza_rating(line_list[2][7]), #"Launch Rating"
                        None, #"Offroad Rating"
                        forza_link(line_list[1][0]) #"Link"
                    ])
                case "Forza Horizon 4":
                    vehicle_data_list.append([
                        game, #"Game"
                        manufacturer, #"Manufacturer"
                        forza_vehicle_name(game, line_list[1]), #"Vehicle Name" 
                        forza_vehicle_name(game, line_list[1], alt = True), #"Vehicle Name Alternative"
                        forza_year(line_list[0][3]), #"Year"
                        forza_how_to_obtain(game, line_list[2][1][0]), #"How to Obtain"
                        forza_value(line_list[2][-7]), #"Value"
                        forza_pi_class( #"Performance Index Class" 
                            game, 
                            pi_number = line_list[2][-1]
                        ), 
                        forza_pi_number(line_list[2][-1]), #"Performance Index Number"
                        None, #"Type"
                        None, #"Era"
                        None, #"Country"
                        forza_rarity(game, line_list[0][1]), #"Rarity"
                        None, #"Rarity Rating"
                        forza_rating(line_list[2][-6]), #"Speed Rating"
                        forza_rating(line_list[2][-4]), #"Acceleration Rating"
                        forza_rating(line_list[2][-2]), #"Braking Rating"
                        forza_rating(line_list[2][-5]), #"Handling Rating"
                        forza_rating(line_list[2][-3]), #"Launch Rating"
                        None, #"Offroad Rating"
                        forza_link(line_list[1][0]) #"Link"
                    ])
                case "Forza Horizon 5":
                    vehicle_data_list.append([
                        game, #"Game"
                        manufacturer, #"Manufacturer"
                        forza_vehicle_name(game, line_list[0][2:4]), #"Vehicle Name" 
                        forza_vehicle_name(game, line_list[0][2:4], alt = True), #"Vehicle Name Alternative"
                        forza_year(line_list[0][4]), #"Year"
                        forza_how_to_obtain(game, line_list[0][-1], line_list[0][7]), #"How to Obtain"
                        forza_value(line_list[0][6]), #"Value"
                        forza_pi_class( #"Performance Index Class" 
                            game, 
                            pi_number = line_list[0][14]
                        ), 
                        forza_pi_number(line_list[0][14]), #"Performance Index Number"
                        None, #"Type"
                        None, #"Era"
                        forza_country(line_list[0][-1], line_list[0][-2]), #"Country"
                        forza_rarity(game, line_list[0][5]), #"Rarity"
                        None, #"Rarity Rating"
                        forza_rating(line_list[0][8]), #"Speed Rating"
                        forza_rating(line_list[0][10]), #"Acceleration Rating"
                        forza_rating(line_list[0][12]), #"Braking Rating"
                        forza_rating(line_list[0][9]), #"Handling Rating"
                        forza_rating(line_list[0][11]), #"Launch Rating"
                        forza_rating(line_list[0][13]), #"Offroad Rating"
                        forza_link(line_list[0][2]) #"Link"
                    ])
                case "Forza Street":
                    vehicle_data_list.append([
                        game, #"Game"
                        manufacturer, #"Manufacturer"
                        forza_vehicle_name(game, line_list[1]), #"Vehicle Name" 
                        forza_vehicle_name(game, line_list[1], alt = True), #"Vehicle Name Alternative"
                        forza_year(line_list[0][2]), #"Year"
                        forza_how_to_obtain(game, line_list[2][2]), #"How to Obtain"
                        None, #"Value"
                        None, #"Performance Index Class"
                        None, #"Performance Index Number"
                        forza_type(game, line_list[2][1]), #"Type"
                        forza_era(game, line_list[0][1]), #"Era"
                        None, #"Country"
                        forza_rarity(game, line_list[2][2]), #"Rarity"
                        None, #"Rarity Rating"
                        None, #"Speed Rating"
                        None, #"Acceleration Rating"
                        None, #"Braking Rating"
                        None, #"Handling Rating"
                        None, #"Launch Rating"
                        None, #"Offroad Rating"
                        forza_link(line_list[1][0]) #"Link"
                    ])
        # Non-Table Vehicles
        if section in list_sections_list and input_line.startswith("*"):
            line_list = forza_line_list_from_list_section(input_line, section)
            manufacturer = difflib.get_close_matches( #"Manufacturer"
                " ".join(line_list[2][1].split()[0:2]) 
                if len(line_list[2]) > 1 
                else " ".join(line_list[2][0].split()[0:2]),  
                forza_unobtainable_vehicle_manufacturers_dict[section], 
                n=1, cutoff=0
            )[0]
            if game in enable_print_list:
                print("\t\t\t", line_list)
            vehicle_data_list.append([
                game, #"Game"
                manufacturer, #"Manufacturer"
                forza_vehicle_name(game, line_list[2]), #"Vehicle Name" 
                forza_vehicle_name(game, line_list[2], alt = True), #"Vehicle Name Alternative"
                forza_year(line_list[1]), #"Year"
                section, #"How to Obtain"
                None, #"Value"
                None, #"Performance Index Class"
                None, #"Performance Index Number"
                None, #"Type"
                None, #"Era"
                None, #"Country"
                None, #"Rarity"
                None, #"Rarity Rating"
                None, #"Speed Rating"
                None, #"Acceleration Rating"
                None, #"Braking Rating"
                None, #"Handling Rating"
                None, #"Launch Rating"
                None, #"Offroad Rating"
                forza_link(line_list[2][0]) #"Link"
            ])